In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

dataset = 'rostos'

if not os.path.exists(dataset):
    os.makedirs(dataset)

detector = MTCNN()
target_size = (160,160)

# extrair rosto de uma única imagem
def extract_face(filename):
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face_arr = cv2.resize(img, target_size)
    return face_arr

# carregar as imagens de rostos, dentro da pasta de uma label
def load_faces(path):
    FACES = []
    for im_name in os.listdir(path):
        try:
            filename = path + im_name
            single_face = extract_face(filename)
            FACES.append(single_face)
        except Exception as e:
            pass
    return FACES

x = []
y = []

# carregar todas as imagens de rostos, dentro de todas as pastas de labels
path = dataset +'/'
FACES = load_faces(path)
labels = [1 for _ in range(len(FACES))]
print(f"1 carregado com sucesso: {len(labels)} rostos.")
x.extend(FACES)
y.extend(labels)

X, Y = np.asarray(x), np.asarray(y)


# modelo de extração de caracteristicas faciais (FaceNet)
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0) 
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)


EMBEDDED_X = []

for img in X:
    EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X = np.asarray(EMBEDDED_X)

1 carregado com sucesso: 223 rostos.
1/1 [==============================] - 0s 73ms/step


In [2]:
EMBEDDED_X.shape

(223, 512)

In [20]:
def erro_medio(y_proba):
    print([i.max() for i in y_proba])
    maior = y_proba.max()
    menor = min([i.max() for i in y_proba])
    return (maior - menor) / maior

In [21]:
def mcsvm(x):
    y = [0 if not i else 1 for i in range(len(x))]

    svm = SVC(kernel='linear', probability=True)
    svm.fit(x, y)
    y_pred = svm.predict_proba(x)
    
    # achar o label 1 com menor probabilidade de ser label 0
    chances = [i[0] for i in y_pred]
    menor = chances.index(min(chances))

    y = [0,1]
    svm.fit([x[0], x[menor]], y)
    y = svm.predict(x)

    print(y)
    print(erro_medio(y_pred))

    # while True:
    #     mudou = False
    #     svm.fit(x, y)
    #     y_pred = svm.predict_proba(x)
    #     for i in range(len(y_pred)):
    #         if y_pred[i][0] > y_pred[i][1]:
    #             y[i] = 0
                


In [22]:
clusters = mcsvm(EMBEDDED_X)

[0 1 1 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 1 1
 0 1 1 1 1 1 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 0 1 0 1 1 0
 0 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1
 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0
 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0
 0]
[0.9295881694351414, 0.9965152757695274, 0.9999847454895894, 0.995999515677589, 0.9923106483533567, 0.9911355943773227, 0.9999897827806109, 0.9951424325672599, 0.9928880301869402, 0.9963697082776699, 0.995237234566499, 0.9964676172668067, 0.9930036596365794, 0.9942697522329477, 0.9943352255278421, 0.9959176473952623, 0.9941731792473335, 0.9944880354619905, 0.9949101076756555, 0.9958181985911584, 0.9950178314043593, 0.9953006839834909, 0.9876672133879977, 0.9968873514527583, 0.9973195426698952, 0.9971043891819695, 0.9957765741155652, 0.99362